In [93]:
import pandas as pd
import numpy as np
import random
import ast
from collections import defaultdict

In [94]:
df = pd.read_csv(R'D:\calorie_RL\data\meal3_mealTime.csv')
df

,meal_name,ingredients,calories_per_serving,carbs,proteins,fats,diet_type,meal_time,serving_size,cost_per_serving_in_inr,region,meal_time_categories
0,Chole Bhature,"Chickpeas, Refined Flour, Spices, Onion",450,55,15.0,18.0,Vegetarian,breakfast/lunch,1 plate,50,North Indian,"['breakfast', 'lunch']"
1,Aloo Paratha,"Wheat Flour, Potato, Spices, Butter",290,35,6.0,12.0,Vegetarian,breakfast/lunch,1 paratha,25,North Indian,"['breakfast', 'lunch']"
2,Butter Chicken,"Chicken, Butter, Cream, Tomato, Spices",490,18,28.0,32.0,Non-Vegetarian,dinner,1 bowl,80,North Indian,['dinner']
3,Kadhi Pakora,"Besan, Curd, Spices, Onion",260,20,8.0,16.0,Vegetarian,lunch,1 bowl,30,North Indian,['lunch']
4,Litti Chokha,"Wheat Flour, Sattu, Brinjal, Potato, Tomato",360,50,10.0,12.0,Vegetarian,snacks,3 littis with chokha,25,North Indian,['snacks']
...,...,...,...,...,...,...,...,...,...,...,...,...
483,Oats Porridge,"Oats, Milk, Sugar/Honey",150,25,5.0,3.0,Vegetarian,Breakfast,1 bowl,30,North Indian,['breakfast']
484,Sweet Potato Chaat,"Sweet Potato, Spices, Lemon Juice",180,30,3.0,5.0,Vegetarian,Snack,1 bowl,30,North Indian,['snacks']
485,"Fruit Salad (Banana, apple, mango,blueberry)","Banana, apple, mango,blueberry",120,30,2.0,1.0,vegetarian,Snack,1 bowl,60,North Indian,['snacks']
486,Basmati Rice,"Basmati Rice, Water",220,45,5.0,2.0,Vegetarian,Lunch,1 cup cooked,30,North Indian,['lunch']


In [95]:

def filter_meals_by_diet(diet_type, meal_time):
    # Ensure 'meal_time_categories' is a list
    def parse_list(value):
        if isinstance(value, str):
            try:
                return ast.literal_eval(value)  # Safely convert string to list
            except (ValueError, SyntaxError):
                return []  # Return an empty list if parsing fails
        return value

    df['meal_time_categories'] = df['meal_time_categories'].apply(parse_list)
    
    # Filter meals by diet_type and meal_time
    filtered_meals = df[(df['diet_type'] == diet_type) & 
                        (df['meal_time_categories'].apply(lambda x: meal_time in x))]
    return filtered_meals
# ...existing code...

In [96]:
# ...existing code...
class DietEnvironment:
    def __init__(self, meals, total_calories, total_budget, meal_times):
        self.meals = meals
        self.total_calories = total_calories
        self.total_budget = total_budget
        self.meal_times = meal_times  # List of meal times (e.g., ['breakfast', 'lunch', 'dinner'])
        self.current_calories = {meal_time: 0 for meal_time in meal_times}
        self.current_budget = 0
        self.selected_meals = {meal_time: [] for meal_time in meal_times}
        self.current_meal_time = 0  # Index to track the current meal time

    def reset(self):
        self.current_calories = {meal_time: 0 for meal_time in self.meal_times}
        self.current_budget = 0
        self.selected_meals = {meal_time: [] for meal_time in self.meal_times}
        self.current_meal_time = 0
        return self.get_state()

    def get_state(self):
        if self.current_meal_time >= len(self.meal_times):
            return (0, self.current_budget)  # Return a default state when out of bounds
        meal_time = self.meal_times[self.current_meal_time]
        return (self.current_calories[meal_time], self.current_budget)

    def step(self, action):
        if self.current_meal_time >= len(self.meal_times):
            return self.get_state(), 0, True  # Return a default state if out of bounds

        meal_time = self.meal_times[self.current_meal_time]
        meal = self.meals.iloc[action]
        calories = meal['calories_per_serving']
        cost = meal['cost_per_serving_in_inr']

        # Add meal to the plan
        self.selected_meals[meal_time].append(meal['meal_name'])
        self.current_calories[meal_time] += calories
        self.current_budget += cost

        # Calculate reward
        if self.current_budget > self.total_budget:
            reward = -50  # Penalty for exceeding budget
            done = True
        elif self.current_calories[meal_time] >= 0.8 * self.total_calories:  # Flexible calorie goal
            reward = 50  # Reward for meeting calorie goal
            self.current_meal_time += 1  # Move to the next meal time
            done = self.current_meal_time >= len(self.meal_times)  # Done if all meal times are covered
        else:
            reward = -5  # Small penalty for not meeting the goal yet
            done = False

        return self.get_state(), reward, done
# ...existing code...

In [97]:
class DietAgent:
    def __init__(self, n_actions):
        self.n_actions = n_actions
        self.q_table = defaultdict(lambda: np.zeros(self.n_actions))
        self.epsilon = 1.0  # Start with high exploration
        self.epsilon_min = 0.1  # Minimum exploration rate
        self.epsilon_decay = 0.995  # Decay rate
        self.alpha = 0.1  # Learning rate
        self.gamma = 0.9  # Discount factor

    def choose_action(self, state):
        state = str(state)
        if random.uniform(0, 1) < self.epsilon:
            return random.randint(0, self.n_actions - 1)  # Explore
        else:
            return np.argmax(self.q_table[state])  # Exploit

    def learn(self, state, action, reward, next_state):
        state = str(state)
        next_state = str(next_state)
        predict = self.q_table[state][action]
        target = reward + self.gamma * np.max(self.q_table[next_state])
        self.q_table[state][action] += self.alpha * (target - predict)

        # Decay epsilon
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [98]:
def train_agent(diet_type, total_calories, total_budget, meal_times, episodes=5000):
    meals_by_time = {meal_time: filter_meals_by_diet(diet_type, meal_time) for meal_time in meal_times}
    if any(meals.empty for meals in meals_by_time.values()):
        raise ValueError(f"No meals found for one or more meal times: {meal_times}")

    env = DietEnvironment(pd.concat(meals_by_time.values()), total_calories, total_budget, meal_times)
    agent = DietAgent(len(env.meals))

    for _ in range(episodes):
        state = env.reset()
        done = False

        while not done:
            action = agent.choose_action(state)
            next_state, reward, done = env.step(action)
            agent.learn(state, action, reward, next_state)
            state = next_state

    return agent

In [ ]:
def generate_diet_plan(agent, diet_type, total_calories, total_budget, meal_times):
    # Filter meals for each meal_time
    meals_by_time = {meal_time: filter_meals_by_diet(diet_type, meal_time) for meal_time in meal_times}
    
    # Check if any meal_time has no available meals
    for meal_time, meals in meals_by_time.items():
        if meals.empty:
            raise ValueError(f"No meals available for {meal_time}. Please check the dataset or constraints.")
    
    # Initialize calorie allocation for each meal_time
    calorie_allocation = {meal_time: total_calories // len(meal_times) for meal_time in meal_times}
    remaining_calories = total_calories % len(meal_times)
    
    # Distribute remaining calories dynamically
    for meal_time in meal_times:
        if remaining_calories > 0:
            calorie_allocation[meal_time] += 1
            remaining_calories -= 1

    # Initialize a dictionary to store the selected meals
    selected_meals_details = {meal_time: [] for meal_time in meal_times}
    total_selected_calories = 0

    # Iteratively select meals for each meal_time
    while total_selected_calories < total_calories:
        for meal_time in meal_times:
            if calorie_allocation[meal_time] <= 0:
                continue  # Skip if calorie allocation for this meal_time is already satisfied
            
            available_meals = meals_by_time[meal_time]
            if available_meals.empty:
                continue  # Skip if no meals are available for this meal_time
            
            # Select a meal that fits within the calorie allocation
            for _, meal in available_meals.iterrows():
                if len(selected_meals_details[meal_time]) < 2:  # Limit to 2 meals per meal_time
                    selected_meals_details[meal_time].append({
                        'meal_name': meal['meal_name'],
                        'calories_per_serving': meal['calories_per_serving'],
                        'cost_per_serving_in_inr': meal['cost_per_serving_in_inr'],
                        'meal_time': meal_time
                    })
                    total_selected_calories += meal['calories_per_serving']
                    calorie_allocation[meal_time] -= meal['calories_per_serving']
                    break  # Move to the next meal_time
            
            # Stop if total calories are satisfied
            if total_selected_calories >= total_calories:
                break

    # Ensure all meal_times have at least one meal
    for meal_time in meal_times:
        if not selected_meals_details[meal_time]:
            print(f"No meal selected for {meal_time}. Attempting to adjust constraints.")
            # Attempt to select up to 2 random meals for the missing meal_time
            available_meals = meals_by_time.get(meal_time, pd.DataFrame())
            if not available_meals.empty:
                random_meals = available_meals.sample(min(2, len(available_meals)))
                for _, random_meal in random_meals.iterrows():
                    selected_meals_details[meal_time].append({
                        'meal_name': random_meal['meal_name'],
                        'calories_per_serving': random_meal['calories_per_serving'],
                        'cost_per_serving_in_inr': random_meal['cost_per_serving_in_inr'],
                        'meal_time': meal_time
                    })
                    total_selected_calories += random_meal['calories_per_serving']
                    if total_selected_calories >= total_calories:
                        break
            else:
                raise ValueError(f"No meals available for {meal_time}. Please check the dataset or constraints.")

    # Flatten the selected meals into a single DataFrame
    diet_plan_df = pd.DataFrame([meal for meals in selected_meals_details.values() for meal in meals])
    return diet_plan_df

In [112]:
diet_type = 'Vegetarian'
total_calories = 2000
total_budget = 500
meal_times = ['breakfast', 'lunch', 'snacks', 'dinner']

# Train the agent and generate the diet plan
agent = train_agent(diet_type, total_calories, total_budget, meal_times)
diet_plan_details = generate_diet_plan(agent, diet_type, total_calories, total_budget, meal_times)

# Combine all meal plans into a single DataFrame
diet_plan_df = pd.DataFrame()

for meal_time, details in diet_plan_details.items():
    # Ensure details is a dictionary and convert it to a DataFrame
    details_df = pd.DataFrame([details])  # Convert the dictionary to a DataFrame
    details_df['meal_time'] = meal_time  # Add a column for meal time
    diet_plan_df = pd.concat([diet_plan_df, details_df], ignore_index=True)

# Display the final DataFrame
print(diet_plan_df)


No meal selected for snacks. Attempting to adjust constraints.
No meal selected for dinner. Attempting to adjust constraints.
                   0          1              2                 3           4  \
0  Daal baati churma     Sheera  Chole Bhature  Mixed Bean Sabzi  Hando Guri   
1                500        250            450               200         180   
2                 80         30             50                35          25   
3          breakfast  breakfast          lunch             lunch      snacks   

              5             6                meal_time  
0  Kuzhakkattai  Kadai paneer                meal_name  
1           160           280     calories_per_serving  
2            30            45  cost_per_serving_in_inr  
3        snacks        dinner                meal_time  


In [113]:
diet_plan_df.T

,0,1,2,3
0,Daal baati churma,500,80,breakfast
1,Sheera,250,30,breakfast
2,Chole Bhature,450,50,lunch
3,Mixed Bean Sabzi,200,35,lunch
4,Hando Guri,180,25,snacks
5,Kuzhakkattai,160,30,snacks
6,Kadai paneer,280,45,dinner
meal_time,meal_name,calories_per_serving,cost_per_serving_in_inr,meal_time
